# Naive Quantum Computer Simulator

In [ ]:
print("Start!")

In [ ]:
import numpy as np
import string

In [ ]:
# Constants

# Constant: e^(i pi / 4) = (1+i) / sqrt(2)
EIPIO4 = (1 + 1j) / np.sqrt(2)

# Constant: 1/sqrt(2)
OOST = 1 / np.sqrt(2)

# A small number
epsilon = 1e-6

In [ ]:
# Define 1-qubit gates

IDENTITY = np.array([[1, 0],[0, 1]])
SIGMA_X = np.array([[0, 1],[1, 0]])
SIGMA_Y = np.array([[0, -1j],[1j, 0]])
SIGMA_Z = np.array([[1, 0],[0, -1]])
HADAMARD = 1/np.sqrt(2) * np.array([[1, 1],[1, -1]])
S_GATE = np.array([[1, 0], [0, 1j]])
T_GATE = np.array([[1, 0], [0, EIPIO4]])
SX_GATE = 1/2 * np.array([[1 + 1j, 1 - 1j], [1 - 1j, 1 + 1j]])

In [ ]:
# Define 2-Qubits gates

CNOT = np.array([
                [[[1, 0], [0, 0]], 
                 [[0, 1], [0, 0]]], 
                [[[0, 0], [0, 1]],
                 [[0, 0], [1, 0]]]
                ])

CZ = np.array([
                [[[1, 0], [0, 0]], 
                 [[0, 1], [0, 0]]],
                [[[0, 0], [1, 0]],
                 [[0, 0], [0, -1]]]
                ])

### Quantum computer class

In [ ]:
MAX_QUBITS = len(string.ascii_lowercase) - 4

class QuantumComputer:
    """
    Implement a simple (and extremelly inneficient) quantum computer simulator
    """
    
    def __init__(self, num_qubits):
        """
        Initialize a quantum computer simulator with 'num_qubits' 
        """
        self.num_qubits = num_qubits
        assert num_qubits > 0, f"Number of Qubits must be positive, got {num_qubits}"
        assert num_qubits < MAX_QUBITS, f"Cannot simulate more than {MAX_QUBITS} Qubits!"
        # Initialize as a tensor of num_qubits. Two dimensions per qubit, so the total size of the vector is 2^n elements
        self.psi = np.zeros([2] * num_qubits, dtype=complex)
        # Set the first element of the (flattened) state to one, this corresponds to state |0000...000>
        self.psi.flat[0] = 1

    def apply_one_qubit_gate(self,U, n):
        """
        Apply unary operator 'U' to single Qubit number 'n'. We use Einstein sums (np.einssum).
        The subscritps are:
            - psi[a, b, c, ...] : Note that we assume there are at most 
            - U[y, z] : We use the last two leters of the alphabet instead of the most standard 'i' and 'j' to allow modeling a few more qubits
        """
        subscripts =  f"yz , {self.eins_vars(n, 'z')} -> {self.eins_vars(n, 'y')}"
        self.psi = np.einsum(subscripts, U, self.psi)

    def apply_two_qubit_gate(self,U, n0, n1):
        """
        Apply unary operator 'U' to two Qubit numbers 'n0' and 'n1'
        For example aplying.a CNOT, n0 is the control qubit and n1 the target qubit
        The subscritps are:
            - psi[a, b, c, ...] : Note that we assume there are at most 
            - U[w, x, y, z] : We use the last four leters of the alphabet
        """
        subscripts =  f"wxyz , {self.eins_vars(n0, 'y', n1, 'z')} -> {self.eins_vars(n0, 'w', n1, 'x')}"
        self.psi = np.einsum(subscripts, U, self.psi)

    def eins_vars(self, n1, char1, n2=None, char2=''):
        """
        Create a string with index names for an einstein sum, replace item 'n1' with character 'char1'
        and 'n2' with character 'char2' (if n2 is non-negative)
        Note: We are assuming that there are at most len(string.ascii_lowercase) Qubits, which is a reasonable limitation for a standard laptop
        """
        assert (n1 >= 0) and (n1 < self.num_qubits), f"Error: Qubit number {n1} out of range"
        assert (n2 is None) or ((n2 >= 0) and (n2 < self.num_qubits)), f"Error: Qubit number {n2} out of range"
        assert (n1 != n2), f"Error: Qubit numbers should be different: n1={n1}, n2={n2}"
        eins = list(string.ascii_lowercase[:self.num_qubits])
        eins[n1] = char1
        if n2 is not None:
            eins[n2] = char2
        return ''.join(eins)

    def cnot(self, n0, n1):
        """ CNOT gate: 'n0' is the control qubit and 'n1' it the target qubit """
        self.apply_two_qubit_gate(CNOT, n0, n1)
        
    def cx(self, n0, n1):
        """ Alias for CNOT """
        self.cnot(n0, n1)

    def cz(self, n0, n1):
        """ CZ gate: 'n0' is the control qubit and 'n1' it the target qubit """
        self.apply_two_qubit_gate(CZ, n0, n1)

    def h(self, n):
        """ Apply Hadamard gate to qubit n """
        self.apply_one_qubit_gate(HADAMARD, n)

    def identity(self, n):
        """ Apply I to qubit n """
        self.apply_one_qubit_gate(IDENTITY, n)

    def _mask_ones(self, n):
        """
        Create a mask cosisting of all 1s except at bit 'n' 
        E.g. for a num_qubits=4
           _mask_ones(0) = 1110
           _mask_ones(1) = 1101
           _mask_ones(2) = 1011
           _mask_ones(3) = 0111
        """
        return ((1 << self.num_qubits) - 1) ^ (1 << (self.num_qubits - n - 1))
    
    def _mask_zeros(self, n):
        """
        Create a bit mask of all zeros, except qubit 'n' 
        E.g. for a num_qubits=4
           _mask_zeros(0) = 0001
           _mask_zeros(1) = 0010
           _mask_zeros(2) = 0100
           _mask_zeros(3) = 1000
        """
        return 1 << (self.num_qubits - n - 1)

    def measure(self, n):
        """
        Measure Qubit number 'n'
        """
        # Calculate a probabilities for qubit 'n'
        p = self.probabilities(n)
        # Measure a random bit according to p[0] and p[1], i.e. Bernoulli distribution
        r = np.random.rand()
        bit = 0 if r <= p[0] else 1
        # Update the system's state after measurement
        self._update_psi(n, bit, p)
        return bit

    def probabilities(self, n):
        """
        Return measurement probabilities 
        Note: Returns an array of two probabilities for measuting states 0 and 1 in standard / computational basis
        """
        subscripts = f"yz , {self.eins_vars(n, 'z')} -> y"
        psi_p = np.square(np.abs(self.psi))
        p = np.einsum(subscripts, IDENTITY, psi_p)
        assert len(p) == 2, f"Error: Probability vector should be length 2, got {p.shape}"
        assert np.abs(1 - np.sum(p)) < epsilon, f"Error: Probabilities don't add up p_0 + p_1 =  {p}\n\tamp = {amp}"
        return p

    def __repr__(self):
        """ Show the system's state """
        # Iterate over the state keeping track of the multi-dimensional index
        it = np.nditer(self.psi, flags=['multi_index']) 
        state = dict()
        for psi_i in it:
            state_str = ''.join([str(q) for q in it.multi_index]) # Convert a tuple index to a string without separation
            state_str = state_str[::-1] # Reverse string
            state[state_str] = psi_i
        s = f"Number of Qubits: {self.num_qubits}\npsi:\n"
        for k in sorted(state.keys()):
            s += f"\t|{k}> : {state[k]}\n" # Show state and value
        return s

    def s(self, n):
        """ Apply S gate to qubit n """
        self.apply_one_qubit_gate(S_GATE, n)

    def state_flat(self):
        """
        Shows the state tensor flattened (in state order)
        Useful for comparing to other programs (e.g. IBM quantum composer's state vector)
        """
        it = np.nditer(self.psi, flags=['multi_index'])
        state = dict()
        for psi_i in it:
            state_str = ''.join([str(q) for q in it.multi_index]) # Convert a tuple index to a string without separation
            state_str = state_str[::-1] # Reverse string
            state[state_str] = psi_i
        psi_flat = np.zeros(self.psi.size, dtype=complex)
        for i, k in enumerate(sorted(state.keys())):
            psi_flat[i] = state[k]
        return psi_flat

    def sx(self, n):
        """ Apply SX gate (a.k.a. square root of X) to qubit n """
        self.apply_one_qubit_gate(SX_GATE, n)

    def t(self, n):
        """ Apply T gate to qubit n """
        self.apply_one_qubit_gate(T_GATE, n)
    
    def _update_psi(self, n, bit, p):
        """ Update state according to measured qubit 'n' with value 'bit' """
        # Set to zero all amplitudes for states that are NOT compatible with our measurement (bit)
        # The psudo code to reset the amplitudes would be:
        #     psi[:, :, :, not_bit, :, :, :] = 0
        #   where 'not_bit' is the opposite of what we measured (i.e. the opposite of `bit` is `1 - bit`)
        idx =  [slice(0, 2)] * self.num_qubits # A list of slices
        idx[n] = 1 - bit # Set index 'n' as the negation of 'bit'
        self.psi[ tuple(idx) ] = 0 # Reset amplitudes
        self.psi = self.psi / np.linalg.norm(self.psi) # Re-normalize psi

    def x(self, n):
        """ Apply X to qubit n """
        self.apply_one_qubit_gate(SIGMA_X, n)

    def y(self, n):
        """ Apply Y to qubit n """
        self.apply_one_qubit_gate(SIGMA_Y, n)

    def z(self, n):
        """ Apply Z to qubit n """
        self.apply_one_qubit_gate(SIGMA_Z, n)

### Random Quantum cirquit generator

This can be used to create test cases

In [ ]:
ALL_GATES = {'x': SIGMA_X, 'y': SIGMA_Y, 'z': SIGMA_Z, 'h': HADAMARD, 's': S_GATE, 't': T_GATE, 'sx': SX_GATE, 'cx': CNOT, 'cz': CZ}

class RandomCircuit:
    """ Create a random quantum circuit """
    def __init__(self, num_qubits, num_gates, seed=None):
        self.num_qubits, self.num_gates = num_qubits, num_gates
        assert num_qubits >= 2
        assert num_gates >= 1
        self.qasm = []
        self.ops = []
        np.random.seed(seed)
        
    def add_random_gate(self):
        gate, gate_name = self.rand_gate()
        l = len(gate.flat)
        if l == 4:
            # Randomly select qubit number
            n = self.rand_qubit_number()
            # Apply 1-Qubit gate
            self.qasm.append(f"{gate_name} q[{n}];")
            self.ops.append(f"qc.{gate_name}({n})")
            self.qc.apply_one_qubit_gate(gate, n)
        elif l == 16:
            # Randomly select qubit numbers
            n0, n1 = -1, -1
            while n0 == n1:
                n0 = self.rand_qubit_number()
                n1 = self.rand_qubit_number()
            # Apply 2-Qubit gate
            self.qasm.append(f"{gate_name} q[{n0}], q[{n1}];")
            self.ops.append(f"qc.{gate_name}({n0}, {n1})")
            self.qc.apply_two_qubit_gate(gate, n0, n1)
        else:
            raise ValueError(f"Unimplemented random gate '{gate_name}'")
        
    def create(self):
        self.qc = QuantumComputer(self.num_qubits)
        self.qasm.append('include "qelib1.inc";');
        self.qasm.append(f"qreg q[{self.num_qubits}];")
        self.qasm.append(f"creg c[{self.num_qubits}];")
        self.ops.append(f"qc = QuantumComputer({self.num_qubits})")
        for i in range(self.num_gates):
            self.add_random_gate()
        return self.qc
            
    def rand_gate(self):
        gate_names = list(ALL_GATES.keys())
        g = np.random.choice(len(gate_names))
        gate_name = gate_names[g]
        return ALL_GATES[gate_name], gate_name
            
    def rand_qubit_number(self):
        return np.random.randint(0, self.num_qubits)

    def __repr__(self):
        qasm = '\n'.join(self.qasm)
        ops = '\n'.join(self.ops)
        return f"QASM:\n{qasm}\n\nOPS:\n{ops}\n\nQC:\n{self.qc}"

# Test cases

In [ ]:
def check_prob_expected(p, p_expected, epsilon=0.000001):
    """ Check expected probabilty vector """
    assert np.min(p) >= 0, f"Error: Probabiltites should be positive, got {p}"
    assert np.max(p) <= 1, f"Error: Probabiltites should be less or equal to 1, got {p}"
    pe = np.reshape(p_expected, p.shape)
    assert np.linalg.norm(p - pe) < epsilon, f"Error checking expected probability:\n\tExpected : {p_expected}\n\tReal     : {p}"

In [ ]:
def check_psi_expected(qc, psi_expected, epsilon=0.001, flatten=False, use_max=False):
    """ Check expected state vector """
    psi = qc.state_flat() if flatten else qc.psi
    pe = np.reshape(psi_expected, psi.shape)
    if use_max:
        diff = np.max(np.abs(psi - pe))
    else:
        diff = np.linalg.norm(psi - pe)
    assert diff < epsilon, f"Error checking expected state:\n\tExpected : {psi_expected}\n\tReal     : {psi}\n\t Diff: {diff}"

In [ ]:
def check_measurement(num_qubits, qc_ops, p_expected, N=10000, epsilon=0.1):
    """
    Check measurements repeteadly after applying a set of operations 'N' times
    """
    shape = [2] * num_qubits
    count = np.zeros(shape)
    for i in range(N):
        qc = QuantumComputer(num_qubits)
        qc_ops(qc)
        idx = [0] * num_qubits
        for i in range(num_qubits):
            bit = qc.measure(i)
            idx[i] = bit
        idx = tuple(idx)
        count[idx] += 1
    p = count / N
    check_prob_expected(p, p_expected, epsilon)

### Test cases: One Qubit gate

In [ ]:
qc = QuantumComputer(1)
qc.identity(0)
check_psi_expected(qc, psi_expected=[1, 0])

In [ ]:
qc = QuantumComputer(1)
qc.x(0)
check_psi_expected(qc, psi_expected=[0, 1])

In [ ]:
qc = QuantumComputer(1)
qc.h(0)
check_psi_expected(qc, psi_expected=[OOST, OOST])

In [ ]:
qc = QuantumComputer(1)
qc.x(0)
qc.z(0)
check_psi_expected(qc, psi_expected=[0, -1])

In [ ]:
qc = QuantumComputer(1)
qc.y(0)
check_psi_expected(qc, psi_expected=[0, 1j])

In [ ]:
qc = QuantumComputer(1)
qc.x(0)
qc.y(0)
check_psi_expected(qc, psi_expected=[-1j, 0])

In [ ]:
qc = QuantumComputer(2)
qc.x(0)
check_psi_expected(qc, psi_expected=[0, 0, 1, 0])

In [ ]:
qc = QuantumComputer(2)
qc.x(1)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0])

In [ ]:
qc = QuantumComputer(2)
qc.x(1)
qc.x(0)
check_psi_expected(qc, psi_expected=[0, 0, 0, 1])

In [ ]:
qc = QuantumComputer(2)
qc.x(0)
qc.x(1)
check_psi_expected(qc, psi_expected=[0, 0, 0, 1])

In [ ]:
qc = QuantumComputer(3)
qc.x(0)
check_psi_expected(qc, psi_expected=[0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
qc = QuantumComputer(3)
qc.x(1)
check_psi_expected(qc, psi_expected=[0, 0, 1, 0, 0, 0, 0, 0])

In [ ]:
qc = QuantumComputer(3)
qc.x(2)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
qc = QuantumComputer(3)
qc.x(0)
qc.x(1)
qc.x(2)
check_psi_expected(qc, psi_expected=[0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
qc = QuantumComputer(3)
qc.h(2)
check_psi_expected(qc, psi_expected=[OOST, OOST, 0, 0, 0, 0, 0, 0])

In [ ]:
qc = QuantumComputer(3)
qc.h(0)
check_psi_expected(qc, psi_expected=[OOST, 0, 0, 0, OOST, 0, 0, 0])

### Test cases: 2-Qubit gates

CNOT using control qubit 0, target qubit 1, i.e.:
```
cnot 0, 1
```


In [ ]:
# CNOT |00> = |00>
qc = QuantumComputer(2)
qc.cnot(0, 1)
check_psi_expected(qc, psi_expected=[1, 0, 0, 0])

In [ ]:
# CNOT |01> = |01>
qc = QuantumComputer(2)
qc.x(1)
qc.cnot(0, 1)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0])

In [ ]:
# CNOT |10> = |11>
qc = QuantumComputer(2)
qc.x(0)
qc.cnot(0, 1)
check_psi_expected(qc, psi_expected=[0, 0, 0, 1])

In [ ]:
# CNOT |11> = |10>
qc = QuantumComputer(2)
qc.x(0)
qc.x(1)
qc.cnot(0, 1)
check_psi_expected(qc, psi_expected=[0, 0, 1, 0])

CNOT using control qubit 1, target qubit 0, i.e.:
```
cnot 1, 0
```


In [ ]:
# CNOT with control 1 and target 0
qc = QuantumComputer(2)
qc.cnot(1, 0)
check_psi_expected(qc, psi_expected=[1, 0, 0, 0])

In [ ]:
# CNOT with control 1 and target 0
qc = QuantumComputer(2)
qc.x(0)
qc.cnot(1, 0)
check_psi_expected(qc, psi_expected=[0, 0, 1, 0])

In [ ]:
# CNOT with control 1 and target 0
qc = QuantumComputer(2)
qc.x(1)
qc.cnot(1, 0)
check_psi_expected(qc, psi_expected=[0, 0, 0, 1])

In [ ]:
# CNOT with control 1 and target 0
qc = QuantumComputer(2)
qc.x(0)
qc.x(1)
qc.cnot(1, 0)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0])

In [ ]:
# Entaglement
qc = QuantumComputer(2)
qc.h(0)
qc.cnot(0, 1)
check_psi_expected(qc, psi_expected=[OOST, 0, 0, OOST])

CNOT with 3 qubits: Control qubit 2, target qubit 1
```
   cnot 2, 1
```

In [ ]:
# CNOT |000> = |000>
qc = QuantumComputer(3)
qc.cnot(2, 1)
check_psi_expected(qc, psi_expected=[1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# CNOT |001> = |011>
qc = QuantumComputer(3)
qc.x(2)
qc.cnot(2, 1)
check_psi_expected(qc, psi_expected=[0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
# CNOT |011> = |001>
qc = QuantumComputer(3)
qc.x(1)
qc.x(2)
qc.cnot(2, 1)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0, 0, 0, 0, 0])

CZ with 3 qubits: Control qubit 2, target qubit 1
```
   cz 2, 1
```

In [ ]:
# CZ |011> = |001>
qc = QuantumComputer(3)
qc.x(1)
qc.x(2)
qc.cz(2, 1)
check_psi_expected(qc, psi_expected=[0, 0, 0, -1, 0, 0, 0, 0])

In [ ]:
# CZ |001> = |001>
qc = QuantumComputer(3)
qc.x(2)
qc.cz(2, 1)
check_psi_expected(qc, psi_expected=[0, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
# CZ |001> = |001>
qc = QuantumComputer(3)
qc.x(0)
qc.x(2)
qc.cz(2, 0)
check_psi_expected(qc, psi_expected=[0, 0, 0, 0, 0, -1, 0, 0])

### Tests: Probabilities

In [ ]:
qc = QuantumComputer(1)
qc.identity(0)
p = qc.probabilities(0)
check_prob_expected(p, [1, 0])

In [ ]:
qc = QuantumComputer(1)
qc.x(0)
p = qc.probabilities(0)
check_prob_expected(p, [0, 1])

In [ ]:
qc = QuantumComputer(1)
qc.h(0)
p = qc.probabilities(0)
check_prob_expected(p, [1/2, 1/2])

In [ ]:
qc = QuantumComputer(2)
qc.x(0)

p0 = qc.probabilities(0)
p1 = qc.probabilities(1)

check_prob_expected(p0, [0, 1])
check_prob_expected(p1, [1, 0])

In [ ]:
qc = QuantumComputer(2)
qc.x(0)
qc.x(1)
p0 = qc.probabilities(0)
p1 = qc.probabilities(1)
check_prob_expected(p0, [0, 1])
check_prob_expected(p1, [0, 1])

In [ ]:
qc = QuantumComputer(2)
qc.h(0)
qc.h(1)
p0 = qc.probabilities(0)
p1 = qc.probabilities(1)
check_prob_expected(p0, [1/2, 1/2])
check_prob_expected(p1, [1/2, 1/2])

### Tets: Measurement

In [ ]:
def qc_ops(qc):
    qc.z(0)

check_measurement(1, qc_ops, p_expected=[1, 0])

In [ ]:
def qc_ops(qc):
    qc.x(0)

check_measurement(1, qc_ops, p_expected=[0, 1])

In [ ]:
def qc_ops(qc):
    qc.x(0)
    qc.x(1)


check_measurement(2, qc_ops, p_expected=[[0, 0], [0, 1]])

In [ ]:
def qc_ops(qc):
    qc.h(0)

check_measurement(1, qc_ops, p_expected=[0.5, 0.5])

In [ ]:
def qc_ops(qc):
    qc.h(0)
    qc.cnot(0, 1)

check_measurement(2, qc_ops, p_expected=[0.5, 0, 0, 0.5])

### Test: Random circuits

Create random circuits and compare them to IBM's Quantum composer's results

In [ ]:
# Random circuit, 2 qubits, 8 operations
qc = QuantumComputer(2)
qc.sx(1)
qc.cx(0, 1)
qc.z(0)
qc.cx(0, 1)
check_psi_expected(qc, psi_expected=[0.5+0.5j, 0+0j, 0.5-0.5j, 0+0j], flatten=True)

In [ ]:
# Random circuit, 2 qubits, 8 operations
qc = QuantumComputer(2)
qc.sx(1)
qc.cx(0, 1)
qc.z(0)
qc.cx(0, 1)
qc.cx(1, 0)
qc.t(0)
qc.y(1)
qc.t(1)

check_psi_expected(qc, psi_expected=[0+0j, 0-0.707j, -0.707+0j, 0+0j], flatten=True)

In [ ]:
# Random circuit, 4 qubits, 32 operations
qc = QuantumComputer(4)
qc.sx(3)
qc.cx(2, 1)
qc.z(2)
qc.cx(0, 3)
qc.cx(3, 2)
qc.t(0)
qc.y(3)
qc.t(1)
qc.s(0)
qc.t(0)
qc.cz(0, 2)
qc.z(3)
qc.sx(3)
qc.cz(2, 0)
qc.z(2)
qc.s(0)
qc.sx(1)
qc.h(0)
qc.y(1)
qc.cz(1, 0)
qc.y(3)
qc.sx(3)
qc.cx(1, 0)
qc.h(1)
qc.cx(3, 1)
qc.t(1)
qc.h(1)
qc.y(1)
qc.y(1)
qc.h(1)
qc.cx(1, 2)
qc.cz(1, 3)

check_psi_expected(qc, psi_expected=[-0.25-0.25j, -0.25+0.25j, 0, 0, 0, 0, -0.354, -0.354j, 0, 0, 0.354, -0.354j, 0.25+0.25j, -0.25+0.25j, 0, 0], flatten=True)

In [ ]:
# Random circuit, 7 qubits, 256 operations
qc = QuantumComputer(7)
qc.sx(3)
qc.cx(6, 1)
qc.z(6)
qc.cx(4, 3)
qc.cx(2, 5)
qc.s(1)
qc.cx(3, 5)
qc.t(1)
qc.s(0)
qc.t(4)
qc.cz(0, 2)
qc.z(3)
qc.sx(3)
qc.cz(2, 4)
qc.z(6)
qc.s(0)
qc.sx(1)
qc.h(0)
qc.y(1)
qc.cz(1, 4)
qc.y(3)
qc.sx(3)
qc.cx(6, 2)
qc.x(3)
qc.y(3)
qc.y(5)
qc.t(5)
qc.h(5)
qc.y(1)
qc.y(1)
qc.h(5)
qc.cx(5, 6)
qc.cz(5, 4)
qc.y(6)
qc.s(1)
qc.cz(3, 4)
qc.cz(4, 6)
qc.x(0)
qc.sx(0)
qc.cx(0, 3)
qc.cx(6, 2)
qc.z(0)
qc.cx(0, 2)
qc.s(1)
qc.sx(1)
qc.cz(3, 6)
qc.cz(3, 1)
qc.x(6)
qc.sx(5)
qc.cx(4, 2)
qc.cx(5, 2)
qc.z(0)
qc.z(4)
qc.z(0)
qc.s(1)
qc.sx(6)
qc.sx(2)
qc.cz(3, 4)
qc.z(6)
qc.sx(0)
qc.h(4)
qc.h(5)
qc.s(6)
qc.sx(4)
qc.h(4)
qc.sx(2)
qc.z(5)
qc.y(1)
qc.cz(4, 5)
qc.h(5)
qc.sx(3)
qc.cz(6, 5)
qc.x(0)
qc.cz(2, 5)
qc.cz(3, 4)
qc.cz(2, 6)
qc.t(2)
qc.cz(5, 4)
qc.x(2)
qc.cx(6, 2)
qc.t(0)
qc.h(0)
qc.x(1)
qc.h(3)
qc.sx(1)
qc.z(0)
qc.s(0)
qc.cx(0, 2)
qc.x(1)
qc.y(3)
qc.t(6)
qc.s(0)
qc.x(2)
qc.y(4)
qc.t(6)
qc.h(6)
qc.cx(2, 0)
qc.t(4)
qc.h(1)
qc.t(5)
qc.x(0)
qc.t(4)
qc.z(3)
qc.h(2)
qc.z(6)
qc.z(3)
qc.sx(3)
qc.cz(0, 6)
qc.y(0)
qc.cz(0, 1)
qc.sx(6)
qc.z(6)
qc.cz(0, 1)
qc.x(5)
qc.s(4)
qc.sx(0)
qc.cz(2, 3)
qc.cx(5, 0)
qc.cx(3, 2)
qc.x(3)
qc.t(3)
qc.z(5)
qc.cz(2, 0)
qc.y(5)
qc.y(1)
qc.t(2)
qc.cz(3, 0)
qc.h(0)
qc.s(3)
qc.cx(6, 2)
qc.x(0)
qc.z(5)
qc.sx(5)
qc.t(5)
qc.z(5)
qc.cx(1, 4)
qc.x(3)
qc.x(4)
qc.z(3)
qc.z(0)
qc.x(3)
qc.s(5)
qc.z(3)
qc.cz(0, 5)
qc.s(6)
qc.z(3)
qc.x(3)
qc.s(6)
qc.x(2)
qc.y(0)
qc.t(1)
qc.z(5)
qc.cx(1, 6)
qc.t(6)
qc.y(6)
qc.y(1)
qc.x(0)
qc.cz(2, 5)
qc.sx(4)
qc.sx(1)
qc.z(1)
qc.cz(4, 3)
qc.sx(0)
qc.h(4)
qc.h(0)
qc.cx(3, 2)
qc.sx(6)
qc.y(1)
qc.sx(6)
qc.t(2)
qc.cz(1, 6)
qc.t(1)
qc.t(0)
qc.h(1)
qc.t(6)
qc.t(2)
qc.s(0)
qc.cx(4, 3)
qc.s(6)
qc.h(5)
qc.h(5)
qc.z(6)
qc.cx(3, 4)
qc.y(3)
qc.z(0)
qc.cx(6, 2)
qc.sx(1)
qc.s(1)
qc.s(5)
qc.sx(6)
qc.cz(3, 4)
qc.x(1)
qc.x(6)
qc.y(5)
qc.t(0)
qc.cx(0, 4)
qc.sx(6)
qc.s(5)
qc.sx(2)
qc.z(4)
qc.t(0)
qc.s(6)
qc.x(5)
qc.h(4)
qc.s(6)
qc.h(0)
qc.s(6)
qc.t(4)
qc.h(1)
qc.h(6)
qc.z(1)
qc.x(4)
qc.h(6)
qc.y(0)
qc.h(3)
qc.y(2)
qc.x(0)
qc.z(2)
qc.s(3)
qc.z(0)
qc.x(1)
qc.y(4)
qc.z(1)
qc.z(6)
qc.x(1)
qc.cx(4, 1)
qc.y(2)
qc.cz(6, 3)
qc.s(1)
qc.cx(3, 0)
qc.s(4)
qc.cz(3, 1)
qc.s(0)
qc.cx(2, 0)
qc.z(3)
qc.y(0)
qc.sx(6)
qc.s(3)
qc.x(6)
qc.sx(0)
qc.z(0)
qc.x(0)
qc.h(0)
qc.t(4)
qc.z(0)
qc.h(3)
qc.cz(2, 3)
qc.cz(6, 3)
qc.h(2)
qc.s(4)
qc.z(3)
qc.cz(3, 2)
qc.s(3)
qc.s(6)
qc.cz(6, 4)
qc.s(1)

check_psi_expected(qc
                   , psi_expected=[ 0.053-0.066j, 0.04-0.009j, 0.035+0.022j, 0.129+0.009j, 0.022+0.035j, 0.009+0.04j, 0.066-0.053j, 0.098+0.085j, 0.022+0.053j, 0.116+0.04j, -0.085+0.079j, 0.053+0.022j, 0.098-0.04j, -0.022+0.035j, -0.072+0.022j, 0.129-0.009j, 0.053+0.085j, 0.066-0.009j, 0.053+0.004j, 0.022-0.098j, 0.085-0.053j, 0.098-0.022j, -0.04-0.035j, -0.009+0.066j, 0.085-0.072j, -0.009-0.022j, 0.022+0.009j, -0.142+0.04j, 0.098+0.022j, -0.085+0.035j, -0.053+0.004j, 0.022-0.116j, 0.04-0.009j, 0.142-0.066j, -0.009-0.147j, -0.022+0.009j, -0.072+0.022j, -0.085-0.009j, -0.066-0.009j, 0.009-0.066j, -0.066-0.009j, 0.098+0.022j, 0.053+0.022j, -0.085-0.009j, 0.035+0.085j, -0.129+0.053j, 0.022+0.035j, 0.009+0.004j, -0.066-0.079j, -0.053-0.085j, 0.053-0.022j, 0.103-0.053j, -0.053+0.04j, 0.022-0.116j, -0.066+0.053j, -0.053-0.066j, -0.022+0.035j, 0.053+0.066j, 0.053-0.04j, -0.022-0.009j, -0.009+0.004j, 0.04+0.098j, -0.129-0.053j, 0.098-0.022j, 0.053-0.11j, 0.022+0.053j, 0.053+0.022j, 0.103+0.053j, 0.085-0.009j, -0.009+0.129j, -0.04+0.098j, 0.009+0.004j, -0.022-0.035j, 0.053-0.066j, -0.085+0.098j, 0.009-0.04j, 0.053-0.066j, 0.066+0.053j, -0.098+0.04j, -0.04-0.009j, 0.009+0.022j, -0.11+0.009j, -0.009+0.147j, -0.022-0.009j, 0.004-0.053j, -0.053-0.04j, -0.129-0.053j, 0.035-0.085j, -0.066+0.009j, 0.098-0.022j, -0.085-0.053j, 0.035-0.04j, 0.009+0.066j, -0.066+0.009j, 0.053-0.085j, 0.085-0.035j, 0.053-0.085j, 0.066+0.009j, -0.066-0.053j, -0.053-0.022j, 0.022+0.116j, -0.053-0.004j, 0.035+0.066j, -0.066-0.053j, -0.053-0.147j, 0.022+0.035j, 0.022-0.009j, -0.142-0.04j, 0.04-0.009j, -0.009-0.004j, 0.098+0.022j, 0.085+0.053j, 0.053+0.066j, 0.04+0.009j, -0.022-0.009j, -0.009-0.066j, -0.129-0.009j, 0.072+0.022j, -0.009-0.066j, 0.066-0.098j, 0.098-0.022j, -0.103+0.009j, -0.085-0.079j, 0.053-0.022j, 0.129+0.053j, 0.053+0.085j, -0.009+0.04j, -0.022+0.035j ]
                   , flatten=True
                   , use_max=True
                  )

In [ ]:
# Random circuit, 6 Qubits, 128 operations
qc = QuantumComputer(6)
qc.x(1)
qc.y(0)
qc.cx(0, 2)
qc.cz(0, 5)
qc.y(3)
qc.h(0)
qc.x(3)
qc.sx(0)
qc.x(2)
qc.y(2)
qc.x(5)
qc.cx(5, 1)
qc.cx(4, 0)
qc.z(2)
qc.h(4)
qc.y(2)
qc.x(5)
qc.s(2)
qc.t(0)
qc.cx(5, 0)
qc.z(4)
qc.x(1)
qc.s(1)
qc.cz(3, 1)
qc.t(3)
qc.z(1)
qc.cx(2, 1)
qc.cx(4, 1)
qc.sx(2)
qc.cx(2, 1)
qc.y(2)
qc.s(3)
qc.x(4)
qc.s(4)
qc.cx(0, 1)
qc.z(2)
qc.t(2)
qc.cz(0, 4)
qc.sx(0)
qc.cz(3, 4)
qc.h(1)
qc.z(3)
qc.z(3)
qc.cx(3, 4)
qc.cx(5, 3)
qc.h(4)
qc.h(2)
qc.cz(1, 3)
qc.z(3)
qc.y(1)
qc.z(2)
qc.s(2)
qc.x(4)
qc.h(0)
qc.h(0)
qc.h(5)
qc.cx(4, 5)
qc.z(0)
qc.x(5)
qc.z(2)
qc.cz(0, 4)
qc.x(3)
qc.x(5)
qc.z(5)
qc.x(0)
qc.t(4)
qc.t(3)
qc.t(4)
qc.cz(2, 1)
qc.y(5)
qc.y(3)
qc.t(4)
qc.y(2)
qc.cz(3, 4)
qc.z(4)
qc.z(5)
qc.s(5)
qc.cz(0, 1)
qc.h(1)
qc.cx(0, 4)
qc.h(3)
qc.y(1)
qc.sx(3)
qc.z(4)
qc.cx(1, 2)
qc.x(1)
qc.cz(2, 4)
qc.x(2)
qc.t(5)
qc.cz(1, 0)
qc.x(3)
qc.z(5)
qc.sx(3)
qc.y(3)
qc.t(4)
qc.cx(2, 3)
qc.x(3)
qc.h(2)
qc.x(1)
qc.y(3)
qc.s(5)
qc.cz(3, 1)
qc.y(3)
qc.sx(1)
qc.cz(4, 5)
qc.s(4)
qc.s(4)
qc.cx(1, 2)
qc.y(2)
qc.z(3)
qc.t(2)
qc.y(5)
qc.cz(3, 2)
qc.h(5)
qc.x(4)
qc.h(1)
qc.x(4)
qc.h(4)
qc.t(0)
qc.s(1)
qc.x(4)
qc.s(4)
qc.t(4)
qc.t(3)
qc.h(0)
qc.z(5)
qc.sx(4)
qc.cx(3, 0)

check_psi_expected(qc
                   , psi_expected=[ -0.044-0.062j, 0.044+0.062j, -0.062+0.018j, -0.062+0.107j, 0.075+0.013j, -0.075-0.013j, -0.031+0.057j, 0.031+0.12j, 0.031+0.12j, -0.031+0.057j, -0.075-0.013j, 0.075+0.013j, 0.107+0.062j, 0.018+0.062j, 0.062-0.044j, -0.062+0.044j, -0.018-0.062j, -0.107-0.062j, -0.062-0.044j, 0.062+0.044j, 0.057+0.031j, 0.12-0.031j, -0.075+0.013j, 0.075-0.013j, 0.075-0.013j, -0.075+0.013j, 0.12-0.031j, 0.057+0.031j, 0.044-0.062j, -0.044+0.062j, -0.062+0.107j, -0.062+0.018j, 0.062-0.018j, 0.062-0.107j, 0.044-0.062j, -0.044+0.062j, -0.031+0.057j, 0.031+0.12j, -0.013-0.075j, 0.013+0.075j, 0.013+0.075j, -0.013-0.075j, 0.031+0.12j, -0.031+0.057j, 0.062+0.044j, -0.062-0.044j, -0.107-0.062j, -0.018-0.062j, 0.187-0.081j, 0.062-0.169j, 0.07-0.187j, -0.195+0.062j, -0.075+0.19j, 0.075+0.164j, -0.083-0.182j, -0.094+0.182j, -0.094+0.182j, -0.083-0.182j, 0.075+0.164j, -0.075+0.19j, 0.062+0.195j, -0.187-0.07j, -0.169-0.062j, -0.081-0.187j ]
                  , flatten=True
                   , use_max=True
                  )

In [ ]:
# Two entangled qubits (Bell state)
qc = QuantumComputer(2)
qc.h(0)
qc.cnot(0, 1)
qc